In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
def cos_theta_layer(theta_vacuum, n):
    return np.sqrt(1 - (np.sin(theta_vacuum) / n) ** 2)

def k_z_layer(theta_vacuum, n, k_0):
    return n * k_0 * cos_theta_layer(theta_vacuum, n)

def phase_layer(theta_vacuum, n, k_0, d):
    k_z_layer_value = k_z_layer(theta_vacuum, n, k_0)
    return k_z_layer_value * d

def eta_layer(n, theta_vacuum, polarization: str):
    cos_theta_layer_value = cos_theta_layer(theta_vacuum, n)
    if polarization == 's':
        return n * cos_theta_layer_value
    elif polarization == 'p':
        return n / cos_theta_layer_value
    else:
        raise ValueError("Polarization must be either 's' or 'p'.")


def layer_characteristic_matrix(n, theta_vacuum, d, k_0, polarization: str):
    # Compute phase and eta arrays (may be scalar or array)
    phase = phase_layer(theta_vacuum, n, k_0, d)
    eta = eta_layer(n, theta_vacuum, polarization)

    # Ensure broadcastable shapes
    phase, eta = np.broadcast_arrays(phase, eta)

    # Construct output matrix of shape (..., 2, 2)
    cos_phase = np.cos(phase)
    sin_term = 1j * np.sin(phase) / eta
    product_term = 1j * np.sin(phase) * eta

    top_row = np.stack([cos_phase, sin_term], axis=-1)
    bottom_row = np.stack([product_term, cos_phase], axis=-1)
    matrix = np.stack([top_row, bottom_row], axis=-2)

    return matrix

def admittance_of_a_matrix(matrix, theta_vacuum, n_substrate, polarization: str):
    eta_substrate = eta_layer(n_substrate, theta_vacuum, polarization)
    output_field = np.array([1, eta_substrate])  # shape: (2,)

    # Expand to shape (..., 2, 1) for broadcasting
    output_field = output_field.reshape((1,) * (matrix.ndim - 2) + (2, 1))

    # Multiply each 2x2 matrix with the output_field column vector
    input_field = matrix @ output_field  # shape: (..., 2, 1)

    admittance = input_field[..., 1, 0] / input_field[..., 0, 0]
    return admittance

def reflectance_of_addmitance(addmitance):
    r = (1 - addmitance) / (1 + addmitance)
    R = np.abs(r) ** 2
    return R

def matrix_power_batch(matrices, power):
    """
    Raise a batch of 2x2 matrices to a given power.
    Input:
        matrices: (..., 2, 2) array
        power: int
    Output:
        (..., 2, 2) array
    """
    shape = matrices.shape[:-2]
    result = np.empty_like(matrices)
    for index in np.ndindex(shape):
        result[index] = np.linalg.matrix_power(matrices[index], power)
    return result


In [12]:


# --- reuse your functions here (cos_theta_layer, k_z_layer, layer_characteristic_matrix, 
#     matrix_power_batch, admittance_of_a_matrix, reflectance_of_addmitance) ---

lambda_0 = 1064e-9  # Wavelength in meters
k_0 = 2 * np.pi / lambda_0  # Wave number in vacuum
n_substrate = 1.76
polarization = 's'
m_stacks = 20

d_1_relative_perturbation = np.linspace(-0.1, 0.1, 101)
d_2_relative_perturbation = np.linspace(-0.1, 0.1, 101)

def plot_reflectance(n_1=1.5, n_2=1.2, theta_deg=5.0):
    theta_vacuum = np.deg2rad(theta_deg)

    d_1 = np.pi / (2 * k_z_layer(theta_vacuum, n_1, k_0)) * (1 + d_1_relative_perturbation)
    d_2 = np.pi / (2 * k_z_layer(theta_vacuum, n_2, k_0)) * (1 + d_2_relative_perturbation)

    D_1, D_2 = np.meshgrid(d_1, d_2, indexing='ij')

    matrix_1 = layer_characteristic_matrix(n_1, theta_vacuum, D_1, k_0, polarization)
    matrix_2 = layer_characteristic_matrix(n_2, theta_vacuum, D_2, k_0, polarization)

    product = matrix_1 @ matrix_2
    product_powered = matrix_power_batch(product, m_stacks)
    matrix = product_powered @ matrix_1

    admittance = admittance_of_a_matrix(matrix, theta_vacuum, n_substrate, polarization)
    R = reflectance_of_addmitance(admittance)
    
    # find index of best reflectance (min 1-R)
    idx_best = np.unravel_index(np.argmin(R), R.shape)
    best_R = R[idx_best]
    best_d1p = d_1_relative_perturbation[idx_best[0]]
    best_d2p = d_2_relative_perturbation[idx_best[1]]
    plt.figure(figsize=(10, 6))
    plt.imshow(R,
               extent=(d_2_relative_perturbation[0], d_2_relative_perturbation[-1],
                       d_1_relative_perturbation[0], d_1_relative_perturbation[-1]),
               origin='lower', aspect='auto', cmap='viridis')
    plt.colorbar(label='Reflectance (R)')
    plt.xlabel('d₂ Relative Perturbation')
    plt.ylabel('d₁ Relative Perturbation')
    plt.title(f"Min Reflectance: {best_R:.2e} at d₁={best_d1p:.3f}, d₂={best_d2p:.3f}")
    plt.show()

# interactive sliders
interact(plot_reflectance,
         n_1=widgets.FloatSlider(value=1.5, min=1.0, max=3.0, step=0.01, description="n₁"),
         n_2=widgets.FloatSlider(value=1.2, min=1.0, max=3.0, step=0.01, description="n₂"),
         theta_deg=widgets.FloatSlider(value=5.0, min=0.0, max=80.0, step=0.5, description="θ (deg)"));


interactive(children=(FloatSlider(value=1.5, description='n₁', max=3.0, min=1.0, step=0.01), FloatSlider(value…